# Import datasets in Neo4j

In [102]:
#!pip install py2neo

In [103]:
from neo4j import GraphDatabase
from py2neo import Graph
from py2neo.data import Node, Relationship
from pprint import pprint
import pandas as pd

## Streamer nodes

In [96]:
streamer_dataset = pd.read_csv('https://raw.githubusercontent.com/gianscuri/Twitch_Community_Graph/main/DataProcessing/Streamer_dataset.csv',
                               index_col = ['streamer']
                              )
streamer_dataset

,idStreamer,minutesLive,viewerMean,viewerPeak,spectMean,spectTot,socialLinks
streamer,,,,,,,
LyonWGFLive,0,1155,14011,19727,9337,58801,https://www.youtube.com/user/WhenGamersFail; h...
Tumblurr,1,2925,13683,32436,10655,148019,NaN
ChristianVieriOfficial,2,570,11014,24931,7056,64614,https://www.instagram.com/christianvieri/; htt...
GrenBaud,3,1005,10285,30017,7600,78521,https://www.tiktok.com/@grenbaud; https://www....
ZanoXVII,4,3480,7681,13973,5699,157718,https://www.instagram.com/zanoxvii/; https://w...
...,...,...,...,...,...,...,...
QLASH_Simracing,2972,180,10,15,5,20,https://www.instagram.com/qlash_global/; https...
teoKrazia,2973,120,10,15,8,21,https://twitter.com/teoKrazia; https://www.ins...
yume940,2974,225,10,23,6,27,https://www.instagram.com/leeyume940/; https:/...


In [44]:
#streamer_dataset.columns = ['idStreamer', 'minutesLive', 'viewerMean', 'viewerPeak', 'spectMean', 'spectTot', 'socialLinks']

In [74]:
streamer_constraint = '''
CREATE CONSTRAINT StreamerNameKey IF NOT EXISTS
FOR (s:Streamer)
REQUIRE s.Name IS UNIQUE
'''

In [75]:
streamer_import_csv = "LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/gianscuri/Twitch_Community_Graph/main/DataProcessing/Streamer_dataset.csv' AS row"
streamer_key = "MERGE (s:Streamer {Name: row.streamer})"

streamer_execution_commands = []
for i in streamer_dataset.columns:
    neo4j_create_statemenet ="FOREACH(ignoreMe IN CASE WHEN trim(row."+ str(i)  + ")" + " <> \"\" THEN [1] ELSE [] END | SET s."+str(i)+" = row."+str(i) +")"
    streamer_execution_commands.append(neo4j_create_statemenet)

streamer_creation_node = '\n'.join(streamer_execution_commands)
streamer_final_query = str(streamer_import_csv)+'\n'+str(streamer_key)+'\n'+str(streamer_creation_node)+'\n'+"RETURN s"
print(streamer_final_query) # bisogna sistemare i nomi delle proprietà che deevono essere conformi alle convenzioni neo4j

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/gianscuri/Twitch_Community_Graph/main/DataProcessing/Streamer_dataset.csv' AS row
MERGE (s:Streamer {Name: row.streamer})
FOREACH(ignoreMe IN CASE WHEN trim(row.idStreamer) <> "" THEN [1] ELSE [] END | SET s.idStreamer = row.idStreamer)
FOREACH(ignoreMe IN CASE WHEN trim(row.minutesLive) <> "" THEN [1] ELSE [] END | SET s.minutesLive = row.minutesLive)
FOREACH(ignoreMe IN CASE WHEN trim(row.viewerMean) <> "" THEN [1] ELSE [] END | SET s.viewerMean = row.viewerMean)
FOREACH(ignoreMe IN CASE WHEN trim(row.viewerPeak) <> "" THEN [1] ELSE [] END | SET s.viewerPeak = row.viewerPeak)
FOREACH(ignoreMe IN CASE WHEN trim(row.spectMean) <> "" THEN [1] ELSE [] END | SET s.spectMean = row.spectMean)
FOREACH(ignoreMe IN CASE WHEN trim(row.spectTot) <> "" THEN [1] ELSE [] END | SET s.spectTot = row.spectTot)
FOREACH(ignoreMe IN CASE WHEN trim(row.socialLinks) <> "" THEN [1] ELSE [] END | SET s.socialLinks = row.socialLinks)
RETURN s


In [53]:
streamer_import_nodes = '''
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/gianscuri/Twitch_Community_Graph/main/DataProcessing/Streamer_dataset.csv' AS row
MERGE (s:Streamer {Name: row.streamer})
FOREACH(ignoreMe IN CASE WHEN trim(row.idStreamer) <> "" THEN [1] ELSE [] END | SET s.idStreamer = row.idStreamer)
FOREACH(ignoreMe IN CASE WHEN trim(row.minutesLive) <> "" THEN [1] ELSE [] END | SET s.minutesLive = row.minutesLive)
FOREACH(ignoreMe IN CASE WHEN trim(row.viewerMean) <> "" THEN [1] ELSE [] END | SET s.viewerMean = row.viewerMean)
FOREACH(ignoreMe IN CASE WHEN trim(row.viewerPeak) <> "" THEN [1] ELSE [] END | SET s.viewerPeak = row.viewerPeak)
FOREACH(ignoreMe IN CASE WHEN trim(row.spectMean) <> "" THEN [1] ELSE [] END | SET s.spectMean = row.spectMean)
FOREACH(ignoreMe IN CASE WHEN trim(row.spectTot) <> "" THEN [1] ELSE [] END | SET s.spectTot = row.spectTot)
FOREACH(ignoreMe IN CASE WHEN trim(row.socialLinks) <> "" THEN [1] ELSE [] END | SET s.socialLinks = row.socialLinks)
RETURN s
'''

## Games nodes

In [76]:
games_dataset = pd.read_csv('https://raw.githubusercontent.com/gianscuri/Twitch_Community_Graph/main/DataProcessing/games_dataset.csv',
                            index_col = ['gameName'])
games_dataset.head()

,idGame,steamName,developer,publisher,releaseDate,categories
gameName,,,,,,
Monster Hunter Generations Ultimate,0,NaN,NaN,NaN,NaN,NaN
Alice: Madness Returns,1,Alice: Madness Returns,Spicy Horse Games,Electronic Arts,2011-06-17,3D Platformer; Action-Adventure; Hack and Slas...
Hold Fast,2,NaN,NaN,NaN,NaN,NaN
Pokémon Trading Card Game Online,3,NaN,NaN,NaN,NaN,NaN
Witch It,4,Witch It,Barrel Roll Games,Daedalic Entertainment,2020-10-22,Multiplayer; Funny; Casual; Action; Indie; Hid...


In [12]:
#games_dataset.columns = ['idGame']

In [77]:
games_constraint = '''
CREATE CONSTRAINT GamesNameKey IF NOT EXISTS
FOR (g:Game)
REQUIRE g.Name IS UNIQUE
'''

In [97]:
games_import_csv = "LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/gianscuri/Twitch_Community_Graph/main/DataProcessing/games_dataset.csv' AS row"
games_key = "MERGE (g:Game {Name: row.gameName})"

games_execution_commands = []
for i in games_dataset.columns:
    neo4j_create_statemenet ="FOREACH(ignoreMe IN CASE WHEN trim(row."+ str(i)  + ")" + " <> \"\" THEN [1] ELSE [] END | SET g."+str(i)+" = row."+str(i) +")"
    games_execution_commands.append(neo4j_create_statemenet)

games_creation_node = '\n'.join(games_execution_commands)
games_final_query = str(games_import_csv)+'\n'+str(games_key)+'\n'+str(games_creation_node)+'\n'+"RETURN g"
print(games_final_query) # bisogna sistemare i nomi delle proprietà che deevono essere conformi alle convenzioni neo4j

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/gianscuri/Twitch_Community_Graph/main/DataProcessing/games_dataset.csv' AS row
MERGE (g:Game {Name: row.gameName})
FOREACH(ignoreMe IN CASE WHEN trim(row.idGame) <> "" THEN [1] ELSE [] END | SET g.idGame = row.idGame)
FOREACH(ignoreMe IN CASE WHEN trim(row.steamName) <> "" THEN [1] ELSE [] END | SET g.steamName = row.steamName)
FOREACH(ignoreMe IN CASE WHEN trim(row.developer) <> "" THEN [1] ELSE [] END | SET g.developer = row.developer)
FOREACH(ignoreMe IN CASE WHEN trim(row.publisher) <> "" THEN [1] ELSE [] END | SET g.publisher = row.publisher)
FOREACH(ignoreMe IN CASE WHEN trim(row.releaseDate) <> "" THEN [1] ELSE [] END | SET g.releaseDate = row.releaseDate)
FOREACH(ignoreMe IN CASE WHEN trim(row.categories) <> "" THEN [1] ELSE [] END | SET g.categories = row.categories)
RETURN g


In [79]:
games_import_nodes = '''
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/gianscuri/Twitch_Community_Graph/main/DataProcessing/games_dataset.csv' AS row
MERGE (g:Game {Name: row.gameName})
FOREACH(ignoreMe IN CASE WHEN trim(row.idGame) <> "" THEN [1] ELSE [] END | SET g.idGame = row.idGame)
FOREACH(ignoreMe IN CASE WHEN trim(row.steamName) <> "" THEN [1] ELSE [] END | SET g.steamName = row.steamName)
FOREACH(ignoreMe IN CASE WHEN trim(row.developer) <> "" THEN [1] ELSE [] END | SET g.developer = row.developer)
FOREACH(ignoreMe IN CASE WHEN trim(row.publisher) <> "" THEN [1] ELSE [] END | SET g.publisher = row.publisher)
FOREACH(ignoreMe IN CASE WHEN trim(row.releaseDate) <> "" THEN [1] ELSE [] END | SET g.releaseDate = row.releaseDate)
FOREACH(ignoreMe IN CASE WHEN trim(row.categories) <> "" THEN [1] ELSE [] END | SET g.categories = row.categories)
RETURN g
'''

## Streamer - streamer relation

In [80]:
streamer_streamer_dataset = pd.read_csv('https://raw.githubusercontent.com/gianscuri/Twitch_Community_Graph/main/DataProcessing/Streamer-Streamer_dataset.csv',
                                        index_col = ['ID_streamer_i', 'ID_streamer_j'])
streamer_streamer_dataset.head()

overlap_percentage
ID_streamer_i ID_streamer_j                    
0             95                           15.0
              129                          13.1
              141                          18.1
              176                          10.2
              182                          14.0

In [91]:
streamer_streamer_rel ='''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/gianscuri/Twitch_Community_Graph/main/DataProcessing/Streamer-Streamer_dataset.csv" AS row
MATCH (i:Streamer {ID_streamer: row.ID_streamer_i}), (j:Streamer {ID_streamer: row.ID_streamer_j})
CREATE (i)-[:HAS_VIEWERSHIP_OVERLAPPED_WITH {overlapPercentage: toFloat(row.overlap_percentage)}]->(j)
'''

## Streamer - games relation

In [82]:
streamer_games_dataset = pd.read_csv('https://raw.githubusercontent.com/gianscuri/Twitch_Community_Graph/main/DataProcessing/Streamer-Games_dataset.csv',
                                     index_col = ['ID_game', 'ID_streamer'])
streamer_games_dataset.head()

minutes
ID_game ID_streamer         
351     80               165
        335               45
        421              120
        471              480
        517               30

In [92]:
streamer_games_rel ='''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/gianscuri/Twitch_Community_Graph/main/DataProcessing/Streamer-Games_dataset.csv" AS row
MATCH (g:Game {ID_game: row.ID_game}), (s:Streamer {ID_streamer: row.ID_streamer})
CREATE (s)-[:PLAYED {minutesPlayed: toInteger(row.minutes)}]->(g)
'''

## Import su Neo4J

In [84]:
data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "twitch"))
session = data_base_connection.session()

In [72]:
#session.run('''MATCH ()-[r] DELETE r''')
#session.run('''MATCH (n) DELETE n''')

In [87]:
session.run(streamer_constraint)
session.run(streamer_import_nodes)

In [89]:
session.run(games_constraint)
session.run(games_import_nodes)

In [94]:
session.run(streamer_streamer_rel)

In [90]:
session.run(streamer_games_rel)

## Possible queries

In [105]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "twitch"))

### Query 1

In [ ]:
query1 = '''
MATCH p=()-->()
RETURN p LIMIT 25
'''

In [106]:
graph.run(query1)
cursor = graph.run(query1)
print(cursor.to_table())

 p                                                                         
---------------------------------------------------------------------------
 (_0)-[:PLAYED {minutesPlayed: 90}]->(_2579)                               
 (_0)-[:PLAYED {minutesPlayed: 105}]->(_2421)                              
 (_0)-[:PLAYED {minutesPlayed: 1440}]->(_3290)                             
 (_0)-[:HAS_VIEWERSHIP_OVERLAPPED_WITH {overlapPercentage: 16.7}]->(_931)  
 (_0)-[:HAS_VIEWERSHIP_OVERLAPPED_WITH {overlapPercentage: 15.8}]->(_1490) 
 (_0)-[:HAS_VIEWERSHIP_OVERLAPPED_WITH {overlapPercentage: 10.3}]->(_2067) 
 (_0)-[:HAS_VIEWERSHIP_OVERLAPPED_WITH {overlapPercentage: 11.7}]->(_452)  
 (_1)-[:PLAYED {minutesPlayed: 1140}]->(_2579)                             
 (_1)-[:HAS_VIEWERSHIP_OVERLAPPED_WITH {overlapPercentage: 43.5}]->(_1454) 
 (_1)-[:HAS_VIEWERSHIP_OVERLAPPED_WITH {overlapPercentage: 10.0}]->(_1595) 
 (_2)-[:PLAYED {minutesPlayed: 165}]->(_2541)                              
 (_2)-[:PLAY

### Query 2

In [115]:
query2 = '''
MATCH p=(s1:Streamer {Name: 'Homyatol'})-[r:HAS_VIEWERSHIP_OVERLAPPED_WITH]-(s2:Streamer)
WHERE r.overlapPercentage > 50
RETURN s2.Name AS Streamer, r.overlapPercentage AS OverlapPercentage
ORDER BY r.overlapPercentage DESC
'''

In [116]:
graph.run(query2)
cursor = graph.run(query2)
print(cursor.to_table())

 Streamer         | OverlapPercentage 
------------------|-------------------
 roccobrazz       |              70.8 
 kajbomb          |              67.0 
 Maadux           |              55.2 
 StupidoHotelShow |              52.7 
 LawayTV          |              51.6 
 panetty          |              51.3 
 VAXPOWER         |              50.8 



### Query 3

In [113]:
query3 = '''
MATCH p=(s:Streamer)-[r:PLAYED]->(g:Game) 
WHERE g.categories CONTAINS 'Open World'
RETURN sum(r.minutesPlayed) AS Totale
'''

In [114]:
graph.run(query3)
cursor = graph.run(query3)
print(cursor.to_table())

 Totale 
--------
 540870 

